In [1]:
import pysam

In [3]:
fn = "/ccb/salz4-3/hji20/off-target-probe-checker/results/bt2/xenium_human_breast_gene_expression_panel_probe_sequences.bam"

In [4]:
amat = dict()
with pysam.AlignmentFile(fn, 'rb') as fh:
    for brec in fh:
        qname = brec.query_name
        if brec.is_unmapped: # highly likely to be intergenic
            print(qname)
        if qname not in amat:
            amat[qname] = [brec.is_reverse]
        else:
            amat[qname].append(brec.is_reverse)

ENSG00000127951|FGL2|e520e9b
ENSG00000127951|FGL2|8b24f03
ENSG00000127951|FGL2|759a825
ENSG00000127951|FGL2|dc67d31
ENSG00000127951|FGL2|73aadf1
ENSG00000198851|CD3E|02369ef
ENSG00000271503|CCL5|e28edbf
ENSG00000166428|PLD4|6696a0b
ENSG00000166428|PLD4|f5bb72b
ENSG00000166428|PLD4|0449862
ENSG00000166428|PLD4|937df62
ENSG00000166428|PLD4|9982460
ENSG00000166428|PLD4|d4621d0
ENSG00000166428|PLD4|3c5cd33
ENSG00000166428|PLD4|caa61e4
ENSG00000166428|PLD4|567ab93
ENSG00000166428|PLD4|34aae6b
ENSG00000166428|PLD4|a0daecf
ENSG00000166428|PLD4|6a3fa57
ENSG00000153563|CD8A|651fabe
ENSG00000153563|CD8A|9d95389
ENSG00000153563|CD8A|0710972
ENSG00000153563|CD8A|da9ac06
ENSG00000132170|PPARG|5d8a10b
ENSG00000132170|PPARG|b660a48
ENSG00000196683|TOMM7|8205364
ENSG00000196683|TOMM7|d2ffb7c
ENSG00000196683|TOMM7|e89014c
ENSG00000196683|TOMM7|6b8d206
ENSG00000169083|AR|a0c6719
ENSG00000168421|RHOH|fe6d4f6


In [5]:
rev_only = []
for qname in amat:
    if False in amat[qname]:
        continue
    else:
        rev_only.append(qname)
print(f'{len(rev_only)} reads with only REV alignments')

2430 reads with only REV alignments


In [6]:
genes = set()
for qname in rev_only:
    gene = qname.split('|')[0]
    genes.add(gene)

In [10]:
genes = set()
for qname in rev_only:
    temp = qname.split('|')
    gene = f'{temp[0]}|{temp[1]}'
    genes.add(gene)

In [11]:
len(genes)

144

In [13]:
for qname in rev_only:
    if "ENSG00000137285" in qname: print(qname)

ENSG00000137285|TUBB2B|86523ef
ENSG00000137285|TUBB2B|950c98d
ENSG00000137285|TUBB2B|2c3eb89
ENSG00000137285|TUBB2B|ec17a71
ENSG00000137285|TUBB2B|2b7cc0a
ENSG00000137285|TUBB2B|fb83179
ENSG00000137285|TUBB2B|4d5e0a1
ENSG00000137285|TUBB2B|1dec8c0


^^ are these all genes on reverse strand?

In [12]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/rev_only_genes.txt"
with open(out_fn, 'w') as fh:
    for x in genes: fh.write(f'{x}\n')

In [14]:
import pandas as pd

fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/gencode.v47.basic.annotation.fmted.gff"
df = pd.read_csv(fn, sep='\t', header=None)
df.columns = ['chr', 'src', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df.head()

,chr,src,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11121.0,24894.0,.,+,.,ID=ENSG00000290825.2;gene_id=ENSG00000290825.2...
1,chr1,HAVANA,transcript,11426.0,14409.0,.,+,.,ID=ENST00000832828.1;Parent=ENSG00000290825.2;...
2,chr1,HAVANA,exon,11426.0,11671.0,.,+,.,ID=exon:ENST00000832828.1:1;Parent=ENST0000083...
3,chr1,HAVANA,exon,12010.0,12227.0,.,+,.,ID=exon:ENST00000832828.1:2;Parent=ENST0000083...
4,chr1,HAVANA,exon,12613.0,12721.0,.,+,.,ID=exon:ENST00000832828.1:3;Parent=ENST0000083...


In [24]:
def get_gid(s):
    temp = s.split(';')
    tid = None
    for x in temp:
        kv = x.split('=')
        if len(kv) != 2: continue
        if kv[0] == 'ID':
            tid = kv[1]
            break
    return tid

ginfos = dict()
for i, row in df.iterrows():
    if row['feature'] == 'gene':
        gid = get_gid(row['attribute']).split('.')[0]
        if not gid: break; print("error parsing gene ID")
        ginfos[gid] = row['strand']

In [42]:
for x in genes:
    if ginfos[x] == '+': print(x); break

^^ all above cases are genes on the reverse strand